Manipulators are verbs

In [ ]:
class Pixel(x: Int, y: Int) {
  def paint(color: Color): Unit = {
    // Implementation here
  }
}

val center = new Pixel(50, 50)
center.paint(new Color("red"))

When an object allows us to manipulate, the name has to be a verb, and there must be no return value.

Examples of refactoring


In [ ]:
class Document {
  def write(content: InputStream): Unit = {
    // Implementation here
  }
}

The example above shows a Document class with a write method that serves as a manipulator. Following the principle, it has a verb name and returns Unit (void).

However, what if we need to know how many bytes were saved? Let's refactor according to the principles:


In [1]:
// Problem: This violates our principle - a manipulator returning a value
class DocumentProblem {
  def write(content: java.io.InputStream): Int = {
    // Implementation that writes and returns bytes written
    42 // Dummy return for example
  }
}

// Solution: Separate the concerns
class Document {
  def output(): OutputPipe = {
    new OutputPipe()
  }
}

class OutputPipe {
  def write(content: java.io.InputStream): Unit = {
    // Implementation here
  }

  def bytes: Int = {
    // Return number of bytes written
    42 // Dummy return for example
  }

  def time: Long = {
    // Return time taken for writing
    123L // Dummy return for example
  }
}

// Usage example
val doc = new Document()
val pipe = doc.output()
pipe.write(new java.io.ByteArrayInputStream("Hello".getBytes))
val bytesWritten = pipe.bytes
val timeElapsed = pipe.time

defined class DocumentProblem
defined class Document
defined class OutputPipe
doc: Document = ammonite.$sess.cmd1$Helper$Document@58e67ec3
pipe: OutputPipe = ammonite.$sess.cmd1$Helper$OutputPipe@3a133558
bytesWritten: Int = 42
timeElapsed: Long = 123L

Notice how we've refactored to maintain the principle:
1. `output()` is a builder - it returns a new object and has a noun name
2. `write()` is a manipulator - it has a verb name and returns Unit
3. `bytes` and `time` are builders - they have noun names and return values


The Builder Pattern and its concerns


In [ ]:
// Builder Pattern example (though the author recommends against it)
class Book private (
  val author: String = "",
  val title: String = "",
  val pages: List[String] = List.empty
) {
  def withAuthor(author: String): Book = {
    new Book(author, title, pages)
  }

  def withTitle(title: String): Book = {
    new Book(author, title, pages)
  }

  def withPage(page: String): Book = {
    new Book(author, title, pages :+ page)
  }
}

// Usage
val myBook = new Book()
  .withAuthor("John Doe")
  .withTitle("Clean Code")
  .withPage("Chapter 1")

The Builder Pattern methods follow the principle - they have noun-like names (with a prefix) and return values. However, as noted in the text, the author generally advises against this pattern as it often leads to less cohesive, larger objects.


A better approach to complex objects with many properties


In [ ]:
// Instead of a large Book class with a builder, break it into smaller objects
class Author(val name: String)
class Title(val text: String)
class Page(val content: String)

class Book(val author: Author, val title: Title, val pages: List[Page]) {
  def printInfo(): Unit = {
    println(s"${title.text} by ${author.name}, ${pages.size} pages")
  }
}

// Usage
val book = new Book(
  new Author("Jane Smith"),
  new Title("Elegant Objects"),
  List(new Page("Introduction"), new Page("Chapter 1"))
)
book.printInfo()

This approach creates smaller, more focused objects rather than using the Builder Pattern for complex initialization.


Avoiding Multiple Return Values


In [ ]:
// In some languages like Go, multiple returns might be used:
// func write(content) (int bytes, int time) { ... }

// In Scala, we isolate the concept in its own class
class Transaction {
  def execute(content: java.io.InputStream): Unit = {
    // Execute the write operation
  }

  def statistics: Statistics = {
    new Statistics(42, 123L) // bytes and time
  }
}

class Statistics(val bytesWritten: Int, val executionTimeMs: Long)

// Usage
val transaction = new Transaction()
transaction.execute(new java.io.ByteArrayInputStream("Data".getBytes))
val stats = transaction.statistics
println(s"Wrote ${stats.bytesWritten} bytes in ${stats.executionTimeMs}ms")

## More Examples of Builders and Manipulators


In [ ]:
// Example: File operations
class File(val path: String) {
  // This is a builder - returns a value, has a noun name
  def content: String = {
    // Read the file content
    scala.io.Source.fromFile(path).mkString
  }

  // This is a manipulator - performs an action, has a verb name, returns Unit
  def rename(newName: String): Unit = {
    // Implementation to rename the file
    println(s"Renamed to $newName")
  }

  // INCORRECT: This violates the principle
  def saveContent(text: String): Boolean = {
    // Implementation that saves and returns success status
    true
  }

  // CORRECT: Refactored approach
  def writer: Writer = new Writer(path)
}

class Writer(path: String) {
  // This is a manipulator - verb name, returns Unit
  def write(content: String): Unit = {
    // Implementation to write content
    println(s"Writing to $path")
  }

  // This is a builder - noun name, returns a value
  def success: Boolean = {
    // Return whether the write was successful
    true
  }
}

// Usage examples
val file = new File("/tmp/example.txt")
val content = file.content // Builder - returns the content

file.rename("newfile.txt") // Manipulator - performs an action

// Incorrect approach - mixing concerns
// val saved = file.saveContent("New text")

// Correct approach - separating concerns
val writer = file.writer
writer.write("New text")
val isSuccess = writer.success


## Why This Principle Matters

This strict separation between builders and manipulators makes code:

1. **More readable** - Method names clearly indicate what they do
2. **More predictable** - Builders don't change state, manipulators don't return values
3. **More maintainable** - Single responsibility principle applied at the method level
4. **More testable** - Easier to test methods with clear, focused responsibilities


## Handling Complex Cases

Sometimes we need both information and manipulation. The solution is to create specific objects for each responsibility:


In [3]:
// Example: Managing a database connection
class Database(url: String) {
  // Builder - returns a connection object
  def connection: Connection = {
    new Connection(url)
  }

  // Builder - returns statistics object
  def stats: Statistics = {
    new Statistics(url)
  }
}

class Connection(url: String) {
  // Manipulator - executes a query
  def execute(sql: String): Unit = {
    println(s"Executing $sql on $url")
  }

  // Manipulator - closes the connection
  def close(): Unit = {
    println("Connection closed")
  }
}

class Statistics(url: String) {
  // Builder - returns query count
  def queryCount: Int = {
    // Implementation to get query count
    42
  }

  // Builder - returns connection time
  def connectionTime: Long = {
    // Implementation to get connection time
    1000L
  }
}

// Usage
val db = new Database("jdbc:postgresql://localhost:5432/mydb")
val conn = db.connection
conn.execute("SELECT * FROM users")
conn.close()

val stats = db.stats
println(s"Executed ${stats.queryCount} queries")
println(s"Connection time: ${stats.connectionTime}ms")


Executing SELECT * FROM users on jdbc:postgresql://localhost:5432/mydb
Connection closed
Executed 42 queries
Connection time: 1000ms


defined class Database
defined class Connection
defined class Statistics
db: Database = ammonite.$sess.cmd3$Helper$Database@34c50ffc
conn: Connection = ammonite.$sess.cmd3$Helper$Connection@58b5f40
stats: Statistics = ammonite.$sess.cmd3$Helper$Statistics@18faf734

## Summary of Section 2.4.2

1. **Manipulators** should:
   - Have verb names (e.g., `save()`, `delete()`, `update()`)
   - Return nothing (Unit in Scala)
   - Change the state of the object or perform an action

2. **Builders** should:
   - Have noun names (e.g., `name()`, `size()`, `content()`)
   - Return a value
   - Not change the state of the object

3. When you need both behaviors, **split the responsibilities** into separate objects or methods, but never mix them in a single method.

4. This approach follows the **Single Responsibility Principle** at the method level, making code more maintainable and easier to understand.
